In [ ]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import scipy.stats as stats
import os
from workalendar.europe import Germany
from datetime import date, timedelta
import numpy as np
import scipy.stats as stats
from statsmodels.tsa.seasonal import seasonal_decompose
import pvlib
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import torch
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings("ignore")

In [ ]:
input_directory = 'E:\TH-Koeln University\Semester 3\Hyperparameter tuning\Dataset\pv_1.csv'

df = pd.read_csv(input_directory, index_col='timestampUtc', parse_dates=True)
df.index = df.index.strftime('%Y-%m-%d %H:%M')
df.index = pd.to_datetime(df.index)
df.head()

In [ ]:
import numpy as np
import pandas as pd

dataset = df.copy()

for i in range(2):
    
    missing_values = dataset['value'].isnull()
    
    for missing_index in dataset.index[missing_values]:
        missing_timestamp = missing_index

        data_2022_date = df.loc[missing_index]

        year = missing_timestamp.year
        month = missing_timestamp.month
        day = missing_timestamp.day
        hour = missing_timestamp.hour
        minute = missing_timestamp.minute    

        if year < 2023:
            year += 1
        elif year >= 2023:
            year -= 1

        adjusted_timestamp = pd.Timestamp(year, month, day, hour, minute)
            
        if adjusted_timestamp in dataset.index:
            adjusted_value = dataset.loc[adjusted_timestamp, 'value']
            dataset.loc[missing_index, 'value'] = adjusted_value

import matplotlib.pyplot as plt
dataset['value'].ffill(inplace=True)
dataset['value'].plot(figsize=(10, 6), title='Value Over Time')
plt.xlabel('Timestamp')
plt.ylabel('Value')
plt.show()


In [ ]:
dataset['group_column'] = 'group_pv_1'   


In [ ]:
input_directory = 'E:\TH-Koeln University\Semester 3\Hyperparameter tuning\Dataset\pv_2.csv'

df2 = pd.read_csv(input_directory, index_col='timestampUtc', parse_dates=True)
df2.index = df2.index.strftime('%Y-%m-%d %H:%M')
df2.index = pd.to_datetime(df2.index)
df2.head()

In [ ]:

dataset2 = df2.copy()

for i in range(2):
    
    missing_values = dataset2['value'].isnull()
    
    for missing_index in dataset2.index[missing_values]:
        missing_timestamp = missing_index

        data_2022_date = df.loc[missing_index]

        year = missing_timestamp.year
        month = missing_timestamp.month
        day = missing_timestamp.day
        hour = missing_timestamp.hour
        minute = missing_timestamp.minute    

        if year < 2023:
            year += 1
        elif year >= 2023:
            year -= 1

        adjusted_timestamp = pd.Timestamp(year, month, day, hour, minute)
            
        if adjusted_timestamp in dataset2.index:
            adjusted_value = dataset2.loc[adjusted_timestamp, 'value']
            dataset2.loc[missing_index, 'value'] = adjusted_value

import matplotlib.pyplot as plt
dataset2['value'].ffill(inplace=True)
dataset2['value'].plot(figsize=(10, 6), title='Value Over Time')
plt.xlabel('Timestamp')
plt.ylabel('Value')
plt.show()


In [ ]:
dataset2['group_column'] = 'group_pv_2'   
dataset2.head()

## Training the model 
https://www.youtube.com/watch?v=9QtL7m3YS9I

https://unit8.com/resources/time-series-forecasting-using-past-and-future-external-data-with-darts/


helpful


https://unit8co.github.io/darts/examples/01-multi-time-series-and-covariates.html

from dartz website: training a global model with multiple covariates (variables or features)

Darts contains many forecasting models, but not all of them can be trained on several time series. The models that support training on multiple series are called global models. An exhaustive list of the global models can be found here (bottom of the table) with for example:

LinearRegressionModel

BlockRNNModel

Temporal Convolutional Networks (TCNModel)

N-Beats (NBEATSModel)

TiDEModel


In [ ]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt

from darts import TimeSeries, concatenate
from darts.utils.callbacks import TFMProgressBar
from darts.utils.timeseries_generation import (
    gaussian_timeseries,
    linear_timeseries,
    sine_timeseries,
)
from darts.models import (
    RNNModel,
    TCNModel,
    TransformerModel,
    NBEATSModel,
    BlockRNNModel,
    VARIMA,
)
from darts.metrics import mape, smape, mae
from darts.dataprocessing.transformers import Scaler
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.datasets import AirPassengersDataset, MonthlyMilkDataset, ElectricityDataset
import pandas as pd
import matplotlib.pyplot as plt
from darts import TimeSeries
from darts.models import ExponentialSmoothing
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from darts import TimeSeries

import warnings

warnings.filterwarnings("ignore")
import logging

logging.disable(logging.CRITICAL)
import logging

logging.disable(logging.CRITICAL)

import warnings

warnings.filterwarnings("ignore")

%matplotlib inline

# for reproducibility
torch.manual_seed(1)
np.random.seed(1)


def generate_torch_kwargs():
    # run torch models on CPU, and disable progress bars for all model stages except training.
    return {
        "pl_trainer_kwargs": {
            "accelerator": "cpu",
            "callbacks": [TFMProgressBar(enable_train_bar_only=True)],
        }
    }

In [ ]:
df = dataset.copy() 
df.reset_index(inplace=True)
df.head()

df2 = dataset2.copy() 
df2.reset_index(inplace=True)
df2.head()

In [ ]:

# Step 1: Prepare Time Series Data
target_column = 'value'  # Replace with the actual column name
series1 = TimeSeries.from_dataframe(df, time_col='timestampUtc', value_cols=target_column)

series1 = series1[-500:]
series1_future = series1[:-500]

covariate_columns = ['t_2mc', 'aswdifd_s', 'aswdir_s', 'vmax_10m', 'tot_prec' ]  # Replace with actual column names
covariate_series_1 = TimeSeries.from_dataframe(df, time_col='timestampUtc', value_cols=covariate_columns)

covariate_series_1 = series1[-500:]
covariate_series_1_future = series1[:-500]


# Step 3: Train-Test Split
train_size = int(len(df) * 0.8)
train_target_series_1, val_target_series_1 = series1[:train_size], series1[train_size:]
train_covariates_series_1, val_covariates_series_1 = covariate_series_1[:train_size], covariate_series_1[train_size:]



In [ ]:

# Step 1: Prepare Time Series Data
target_column = 'value'  # Replace with the actual column name
series2 = TimeSeries.from_dataframe(df2, time_col='timestampUtc', value_cols=target_column)

series2 = series2[-500:]
series2_future = series2[:-500]

covariate_columns = ['t_2mc', 'aswdifd_s', 'aswdir_s', 'vmax_10m', 'tot_prec' ]  # Replace with actual column names
covariate_series_2 = TimeSeries.from_dataframe(df2, time_col='timestampUtc', value_cols=covariate_columns)

covariate_series_2 = series1[-500:]
covariate_series_2_future = series1[:-500]


# Step 3: Train-Test Split
train_size = int(len(df2) * 0.8)
train_target_series_2, val_target_series_2 = series2[:train_size], series2[train_size:]
train_covariates_series_2, val_covariates_series_2 = covariate_series_1[:train_size], covariate_series_2[train_size:]



In [ ]:
# 2 days 4*24*2=192
model_name = "BlockRNN_test"
model_pastcov = BlockRNNModel(
    model="LSTM",
    input_chunk_length=500,
    output_chunk_length=200,
    n_epochs=3,
    random_state=0,
    model_name=model_name,
    save_checkpoints=True,  # store model states: latest and best performing of validation set
    force_reset=True,
    **generate_torch_kwargs()
)

In [ ]:
model_pastcov.fit(
    series=[train_target_series_1],
    past_covariates=[train_covariates_series_1],
    val_series=[val_target_series_1],
    val_past_covariates=[val_covariates_series_1],
)

In [ ]:
model_pastcov = BlockRNNModel.load_from_checkpoint(model_name=model_name, best=True)


### Forecasting 

In [ ]:
# Select a chunk of covariate_series_1 for a specific prediction period
start_time_prediction = series1.end_time() - pd.Timedelta(days=50)  # Adjust the number of days as needed
selected_covariates_chunk = covariate_series_1.slice(start_time_prediction, series1.end_time())

# Now use selected_covariates_chunk as past covariates in the prediction
#pred_cov = model_pastcov.predict(n=36, series=series1, past_covariates=selected_covariates_chunk)


In [ ]:
pred_cov = model_pastcov.predict(n=200, series=series1, past_covariates=covariate_series_1_future)

series1[-100:].plot(label="actual")
pred_cov[-100:].plot(label="forecast")

plt.legend()
plt.show()

In [ ]:
# Trained with past covariates and predicting with future covatiates
pred_cov = model_futcov.predict(n=200, series=series1, future_covariates=covariate_series_1)

series1[-100:].plot(label="actual")
pred_cov[-100:].plot(label="forecast")

plt.legend()
plt.show()

# ------ End ------

# Something about multivariate

In [ ]:
combined_df = pd.concat([df, df2], ignore_index=True)

In [ ]:
print(combined_df)
series_multi = TimeSeries.from_group_dataframe(
    combined_df,
    time_col="timestampUtc",
    group_cols="group_column",  # individual time series are extracted by grouping `df` by `group_cols`
    value_cols=None,  # optionally, specify the time varying columns
    fill_missing_dates=True,
    freq='15min'
)

print(f"\n{len(series_multi)} series were extracted from the input DataFrame")
for i, ts in enumerate(series_multi):
    print(f"Static covariates of series {i}")
    print(ts.static_covariates)
    ts['value'].plot(label=f"comp1_series_{i}")